# Contributing code and models

- Step 1: **Read the [contribution guide](https://ntc-rosetta.readthedocs.io/en/latest/CONTRIBUTING.html)**!
- Check the [support matrix](https://ntc-rosetta.readthedocs.io/en/latest/models/matrix/index.html).
- Read the [NTC-Rosetta documentation](https://ntc-rosetta.readthedocs.io/en/latest/)
- Read the [Yangify documentation](https://yangify.readthedocs.io/en/latest/index.html)
- Read the [Yangson documentation](https://yangson.labs.nic.cz/)
- Read the [RFC's](https://en.wikipedia.org/wiki/YANG)

### Goals of this guide
- A *high level* overview of
  * Adding a YANG library
  * Adding models
  * Adding drivers
  * Writing tests

### Non-goals
- [How to open a PR](https://github.com/networktocode/ntc-rosetta/pulls)
- [How to file a bug](https://github.com/networktocode/ntc-rosetta/issues)
- [How to use Rosetta](https://ntc-rosetta.readthedocs.io/en/latest/tutorials/index.html)
- [How to YANG](https://en.wikipedia.org/wiki/YANG)

## Add a new library or model

The network automation realm is in a constant state of flux, and YANG models are no different with revisions and additions.  First, please double check if the library model you want to support already exists in the [support matrix](https://ntc-rosetta.readthedocs.io/en/latest/models/matrix/index.html).

### Add a library
- Create the appropriate directory structure under `ntc-rosetta/ntc_rosetta/yang`.
- Be sure to include your [YANG library](https://tools.ietf.org/html/rfc8525) json.
- Update the datamodels in `ntc-rosetta/ntc_rosetta/yang/__init__.py`
  - Create a `get_my_new_library` method and add it to the `get_data_model` method.

### Add a model
- Put your file in the appropriate directory under the `ntc-rosetta/ntc_rosetta/yang/` folder.
- Update the [YANG library](https://tools.ietf.org/html/rfc8525) json.

## Add a new driver

- Create a file named after your driver in `ntc-rosetta/ntc_rosetta/drivers/my_awesome_driver.py`
  - This file will import the parsers and translators for each YANG library it supports.  See the [existing drivers](https://github.com/networktocode/ntc-rosetta/tree/develop/ntc_rosetta/drivers) for examples.
    - Each YANG library will have its own driver class in the driver file.  The library-specific parsers and translators are registered to its respective class. 
- Import the new driver in `ntc-rosetta/ntc_rosetta/__init__.py` and register it in the `mapping` dict under `get_driver`.

### Creating parsers and translators

This is the directory layout used within the `ntc_rosetta` module.
```bash
ntc_rosetta/{parsers,translators}/{yang_library_name}
├── __init__.py  # <- probably empty
└── {driver_name}
   ├── __init__.py  # <- Needs to import the parsers/translators from each model
   ├── {model_1_name}
   │  ├── __init__.py  # <- probably empty
   │  └── {attrib}.py  # <- Implements the Yangify parser/translator for {model_1_name}
   └── {model_2_name}
      ├── __init__.py  # <- probably empty
      └── {attrib}.py  # <- Implements the Yangify parser/translator for {model_2_name}

```
- Create the appropriate file structure and write your parsers and translators.  Again, see the [source code](https://github.com/networktocode/ntc-rosetta/tree/develop/ntc_rosetta) for examples.
- As mentioned earlier, each library gets its own class under the master driver, so each `{library_name}Parser` (or Translator) class needs to be registered with [`ntc-rosetta/ntc_rosetta/parsers/__init__.py`](https://github.com/networktocode/ntc-rosetta/blob/develop/ntc_rosetta/parsers/__init__.py).

## Testing

There is a reasonably functioning "dummy" driver complete with tests at `https://github.com/dgjustice/ntc-rosetta/tree/dummy`.  This can be diffed against the origin at commit 994faaf1dbe894360477c3f55751587d82c5d368.  Test structure and code are constantly subject to change, so bear in mind that this is a snapshot in time and may not reflect the most recent changes to the project or any of its dependencies.  The [Makefile](https://github.com/networktocode/ntc-rosetta/blob/develop/Makefile) is the best sourse of truth for what is run during the test cycle.  

#### Documentation

Sphinx is run against the docs, so you will need to update the appropriate index.rst file ([example](https://github.com/networktocode/ntc-rosetta/blob/develop/docs/tutorials/index.rst)).  All jupyter notebooks are also run as part of the CI process, so additions and modifications need to run error-free.

#### Static analysis

Refer to the Makefile and RTFM for the appropriate projects.  MyPy is utilized as well and has excellent [documentation](https://mypy.readthedocs.io/en/latest/index.html).  A coverage report is generated locally by pytest, but it does not appear that this is being collected upstream just yet.

### Writing tests

Test cases are automatically collected based on a pre-determined file structure.  Refer to the test runner [source code](https://github.com/networktocode/ntc-rosetta/blob/develop/tests/models/test_models.py) for details.  Updating tests for existing models and drivers should be fairly self-explanatory if you refer to the source.  For new models and drivers, the setup is a little more involved.  The following structure must exist at a minimum and all the files need to be populated appropriately.  

```bash
ntc-rosetta/tests/models/{org_name}
└── data
   └──{yang_model_name}
      ├── merge
      │  └── {driver_1_name}
      │     ├── add_{obj_name}
      │     │  ├── data_candidate.json
      │     │  ├── data_running.json
      │     │  ├── res_merge
      │     │  └── res_replace
      │     └── remove_{obj_name}
      │        ├── data_candidate.json
      │        ├── data_running.json
      │        ├── res_merge
      │        └── res_replace
      ├── parse
      │  └── {driver_1_name}
      │     └── config
      │        ├── dev_conf
      │        └── result.json
      └── translate
         └── {driver_1_name}
            └── test_case_1
               ├── data.json
               ├── res_merge
               └── res_replace
```  

To make this a little quicker, you can run (using ntc-vlan as an example):  

```bash
#!/usr/bin/env bash
ORG_NAME=ntc
YANG_MODEL_NAME='ntc-vlan'
DRIVER_NAME=ios
OBJ_NAME=vlans
mkdir -p tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/{merge,parse,translate}
mkdir -p tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/merge/$DRIVER_NAME/{"add_$OBJ_NAME","remove_$OBJ_NAME"}
touch tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/merge/$DRIVER_NAME/{"add_$OBJ_NAME","remove_$OBJ_NAME"}/{data_candidate.json,data_running.json,res_merge,res_replace}
mkdir -p tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/parse/$DRIVER_NAME/config
touch tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/parse/$DRIVER_NAME/config/{dev_conf,result.json}
mkdir -p tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/translate/$DRIVER_NAME/test_case_1
touch tests/models/$ORG_NAME/data/$YANG_MODEL_NAME/translate/$DRIVER_NAME/test_case_1/{data.json,res_merge,res_replace}
```  

`make tests` runs the entire suite, so it may be helpful to speed up debugging by simply running `make pytest`.  